In [1]:
#### Making this notebook to try to focus on pydeck viz and separate the memory load.

In [2]:
import pandas as pd
import geopandas as gpd
import geojson

import pydeck



pd.set_option('display.max_columns', None) ### This line makes all the columns display, rather than ellipses shorten

In [3]:
### Import the data

tax_dataframe = gpd.read_file('C:/Users/aaron/Fun Mapping Work/Data/Georgism_Study/Parcels_for_King_County_with_Address_with_Property_Information___parcel_address_area')

## downloading so I can spatial join and trim the tax dataset
seattle_city_limits = gpd.read_file('C:/Users/aaron/Fun Mapping Work/Data/Georgism_Study/seattle_city-limits.geojson')

In [4]:
tax_dataframe = tax_dataframe.to_crs(4326)

tax_dataframe_trimmed = gpd.clip(tax_dataframe, seattle_city_limits)

## Get rid of every row where the landval is suspiciously small
tax_dataframe_trimmed = tax_dataframe_trimmed[~(tax_dataframe_trimmed['APPRLNDVAL'] <= 1100)]  

In [5]:
display_column = 'APPR_IMPR'

In [6]:
#### Make Tax Per SQFT columns for both improved value and land value

#tax_dataframe_trimmed['Land_Val_Per_SQFT'] = tax_dataframe_trimmed['APPRLNDVAL'] / tax_dataframe_trimmed['Shape_Area']
#tax_dataframe_trimmed['Improved_Val_Per_SQFT'] = tax_dataframe_trimmed['APPR_IMPR'] / tax_dataframe_trimmed['Shape_Area']

#tax_dataframe_trimmed['Land_Taxable_Per_SQFT'] = tax_dataframe_trimmed['TAX_LNDVAL'] / tax_dataframe_trimmed['Shape_Area']
#tax_dataframe_trimmed['Improved_Taxable_Per_SQFT'] = tax_dataframe_trimmed['TAX_IMPR'] / tax_dataframe_trimmed['Shape_Area']

#tax_dataframe_trimmed['Land_plus_Imp_Appraised_Per_SQFT'] = tax_dataframe_trimmed['Land_Val_Per_SQFT'] + tax_dataframe_trimmed['Improved_Val_Per_SQFT']
#tax_dataframe_trimmed['Land_plus_Imp_Taxable_Per_SQFT'] = tax_dataframe_trimmed['Land_Taxable_Per_SQFT'] + tax_dataframe_trimmed['Improved_Taxable_Per_SQFT']

#tax_dataframe_trimmed['Land_V_Imp_Appraisal_Ratio'] = tax_dataframe_trimmed['APPR_IMPR'] / tax_dataframe_trimmed['APPRLNDVAL']
#tax_dataframe_trimmed['Land_V_Imp_Taxable_Ratio'] = tax_dataframe_trimmed['APPR_IMPR'] / tax_dataframe_trimmed['APPRLNDVAL']

In [7]:
Land_Val_col_df = tax_dataframe_trimmed[[f'{display_column}', 'LON', 'LAT', 'geometry']]

In [8]:
centroid_frame = Land_Val_col_df.copy()

centroid_column = centroid_frame['geometry'].centroid

centroid_frame['geometry'] = centroid_column

C:\Users\aaron\AppData\Local\Temp\ipykernel_45948\1628008089.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid_column = centroid_frame['geometry'].centroid


In [12]:
#### pydeck
color_and_height_column = display_column

x = 85

pydeck_layer = pydeck.Layer(
    type = "HexagonLayer", 
    data = centroid_frame, 
    get_position=["LON", "LAT"],

    radius = 500, #radius of the hexagons in meters
    get_elevation_weight = color_and_height_column,
    get_color_weight = color_and_height_column,
    
    auto_highlight=True,
    elevation_scale=50,
    pickable=True,
    elevation_range=[0, 500],
    extruded=True,
    coverage=1,
)

view_state = pydeck.ViewState(
    longitude=-122.415,
    latitude=47.6323,
    zoom=9,
    min_zoom=5,
    max_zoom=15,
    pitch=29.5,
    #bearing=-27.36,
    bearing=-7
)

# Render
land_val_viz = pydeck.Deck(
    layers=[pydeck_layer], 
    initial_view_state=view_state,
    tooltip={
        'html': '<b>Total Value of improvements made in this hexagon:</b> {elevationValue}',
        'style': {
            'color': 'white'
        }
    }
)
#land_val_viz

In [10]:
#type (land_val_viz)

pydeck.bindings.deck.Deck

In [11]:
#land_val_viz.to_html('improved_val_hex_viz_500.html') ## this line saves the file as an HTML